In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [2]:
df=pd.read_csv("Train_Test_IoT_GPS_Tracker.csv")

In [3]:
df

,ts,date,time,latitude,longitude,label,type
0,1556242299,25-Apr-19,18:31:39,116.521704,132.162504,1,ddos
1,1556242301,25-Apr-19,18:31:41,121.742933,135.072500,1,ddos
2,1556242306,25-Apr-19,18:31:46,121.795929,135.110198,1,ddos
3,1556242309,25-Apr-19,18:31:49,116.611216,132.224555,1,ddos
4,1556242311,25-Apr-19,18:31:51,121.841526,135.151362,1,ddos
...,...,...,...,...,...,...,...
58955,1554202928,2-Apr-19,4:02:08,1.028628,10.810522,0,normal
58956,1554202928,2-Apr-19,4:02:08,0.345361,10.727989,0,normal
58957,1554202928,2-Apr-19,4:02:08,0.423384,10.508036,0,normal
58958,1554202928,2-Apr-19,4:02:08,0.445292,10.381356,0,normal


In [4]:
encodingLabels=pd.get_dummies(df.type)
df=pd.concat([df, encodingLabels], axis=1)

In [5]:
df

,ts,date,time,latitude,longitude,label,type,backdoor,ddos,injection,normal,password,ransomware,scanning,xss
0,1556242299,25-Apr-19,18:31:39,116.521704,132.162504,1,ddos,0,1,0,0,0,0,0,0
1,1556242301,25-Apr-19,18:31:41,121.742933,135.072500,1,ddos,0,1,0,0,0,0,0,0
2,1556242306,25-Apr-19,18:31:46,121.795929,135.110198,1,ddos,0,1,0,0,0,0,0,0
3,1556242309,25-Apr-19,18:31:49,116.611216,132.224555,1,ddos,0,1,0,0,0,0,0,0
4,1556242311,25-Apr-19,18:31:51,121.841526,135.151362,1,ddos,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58955,1554202928,2-Apr-19,4:02:08,1.028628,10.810522,0,normal,0,0,0,1,0,0,0,0
58956,1554202928,2-Apr-19,4:02:08,0.345361,10.727989,0,normal,0,0,0,1,0,0,0,0
58957,1554202928,2-Apr-19,4:02:08,0.423384,10.508036,0,normal,0,0,0,1,0,0,0,0
58958,1554202928,2-Apr-19,4:02:08,0.445292,10.381356,0,normal,0,0,0,1,0,0,0,0


In [6]:
X=df[["longitude","latitude"]]
y=df[['normal','backdoor','ddos', 'password',  'injection', 'ransomware',  'xss' ,'scanning']]

In [7]:
from sklearn.preprocessing import StandardScaler
 

 
# Initialise the Scaler
scaler = StandardScaler()
 
# To scale data
X=scaler.fit_transform(X)

X

array([[ 1.0038727 ,  0.93420817],
       [ 1.04265624,  1.00407152],
       [ 1.04315867,  1.00478064],
       ...,
       [-0.61750105, -0.61926088],
       [-0.61918939, -0.61896775],
       [-0.61966083, -0.6177983 ]])

In [8]:
y

,normal,backdoor,ddos,password,injection,ransomware,xss,scanning
0,0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
58955,1,0,0,0,0,0,0,0
58956,1,0,0,0,0,0,0,0
58957,1,0,0,0,0,0,0,0
58958,1,0,0,0,0,0,0,0


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)

print("Train Data size:", len(X_train))
print("Test Data size", len(X_test))

Train Data size: 44220
Test Data size 14740


In [10]:
lstm_output_size=100

In [11]:
cnn = Sequential()
cnn.add(Convolution1D(64, 3, padding="same",activation="relu",input_shape=(2, 1)))
cnn.add(Convolution1D(64, 3, padding="same", activation="relu"))
cnn.add(MaxPooling1D(pool_size=(2),padding="same"))
cnn.add(Convolution1D(128, 3, padding="same", activation="relu"))
cnn.add(Convolution1D(128, 3, padding="same", activation="relu"))
cnn.add(MaxPooling1D(pool_size=(2),padding="same"))
cnn.add(LSTM(lstm_output_size))
cnn.add(Dropout(0.1))
cnn.add(Dense(8, activation="softmax"))

In [12]:
cnn.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

In [13]:
checkpointer = callbacks.ModelCheckpoint(filepath="checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('cnntrainanalysisGPSwithScaling.csv',separator=',', append=False)

In [14]:
import time
start = time.time()

history=cnn.fit(X_train, y_train,batch_size=256,epochs=500,validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("cnn_modelGPSwithScaling.hdf5")

end = time.time()
diff = (end - start)/60
print('Training time (in Minutes):', diff)

Epoch 1/500
173/173 [==============================] - 6s 17ms/step - loss: 1.1468 - accuracy: 0.6059 - val_loss: 0.8865 - val_accuracy: 0.6448
Epoch 2/500
173/173 [==============================] - 2s 13ms/step - loss: 0.8211 - accuracy: 0.6464 - val_loss: 0.7782 - val_accuracy: 0.6646
Epoch 3/500
173/173 [==============================] - 2s 11ms/step - loss: 0.7891 - accuracy: 0.6595 - val_loss: 0.7677 - val_accuracy: 0.6849
Epoch 4/500
173/173 [==============================] - 2s 12ms/step - loss: 0.7711 - accuracy: 0.6728 - val_loss: 0.7535 - val_accuracy: 0.6725
Epoch 5/500
173/173 [==============================] - 2s 12ms/step - loss: 0.7623 - accuracy: 0.6752 - val_loss: 0.7348 - val_accuracy: 0.6858
Epoch 6/500
173/173 [==============================] - 2s 11ms/step - loss: 0.7537 - accuracy: 0.6765 - val_loss: 0.7394 - val_accuracy: 0.6812
Epoch 7/500
173/173 [==============================] - 2s 11ms/step - loss: 0.7519 - accuracy: 0.6806 - val_loss: 0.7257 - val_accuracy:

Epoch 28/500
173/173 [==============================] - 2s 14ms/step - loss: 0.5274 - accuracy: 0.7900 - val_loss: 0.4962 - val_accuracy: 0.8026
Epoch 29/500
173/173 [==============================] - 2s 13ms/step - loss: 0.5325 - accuracy: 0.7886 - val_loss: 0.5019 - val_accuracy: 0.8015
Epoch 30/500
173/173 [==============================] - 2s 13ms/step - loss: 0.5152 - accuracy: 0.7959 - val_loss: 0.4918 - val_accuracy: 0.8060
Epoch 31/500
173/173 [==============================] - 2s 14ms/step - loss: 0.5086 - accuracy: 0.7976 - val_loss: 0.4699 - val_accuracy: 0.8154
Epoch 32/500
173/173 [==============================] - 2s 14ms/step - loss: 0.5036 - accuracy: 0.7993 - val_loss: 0.4821 - val_accuracy: 0.8066
Epoch 33/500
173/173 [==============================] - 3s 15ms/step - loss: 0.4953 - accuracy: 0.8020 - val_loss: 0.4939 - val_accuracy: 0.8003
Epoch 34/500
173/173 [==============================] - 3s 16ms/step - loss: 0.4844 - accuracy: 0.8088 - val_loss: 0.4569 - val_ac

173/173 [==============================] - 3s 18ms/step - loss: 0.4122 - accuracy: 0.8379 - val_loss: 0.3966 - val_accuracy: 0.8440
Epoch 55/500
173/173 [==============================] - 3s 18ms/step - loss: 0.4134 - accuracy: 0.8368 - val_loss: 0.3843 - val_accuracy: 0.8459
Epoch 56/500
173/173 [==============================] - 3s 18ms/step - loss: 0.4132 - accuracy: 0.8361 - val_loss: 0.3877 - val_accuracy: 0.8439
Epoch 57/500
173/173 [==============================] - 3s 18ms/step - loss: 0.3991 - accuracy: 0.8439 - val_loss: 0.3969 - val_accuracy: 0.8386
Epoch 58/500
173/173 [==============================] - 3s 16ms/step - loss: 0.4013 - accuracy: 0.8419 - val_loss: 0.3839 - val_accuracy: 0.8473
Epoch 59/500
173/173 [==============================] - 3s 15ms/step - loss: 0.3985 - accuracy: 0.8428 - val_loss: 0.3713 - val_accuracy: 0.8558
Epoch 60/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3969 - accuracy: 0.8439 - val_loss: 0.3793 - val_accuracy: 0.847

Epoch 108/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3531 - accuracy: 0.8642 - val_loss: 0.3263 - val_accuracy: 0.8781
Epoch 109/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3515 - accuracy: 0.8640 - val_loss: 0.3212 - val_accuracy: 0.8754
Epoch 110/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3487 - accuracy: 0.8653 - val_loss: 0.3565 - val_accuracy: 0.8611
Epoch 111/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3549 - accuracy: 0.8626 - val_loss: 0.3501 - val_accuracy: 0.8604
Epoch 112/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3550 - accuracy: 0.8620 - val_loss: 0.3882 - val_accuracy: 0.8454
Epoch 113/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3541 - accuracy: 0.8620 - val_loss: 0.3579 - val_accuracy: 0.8596
Epoch 114/500
173/173 [==============================] - 2s 14ms/step - loss: 0.3489 - accuracy: 0.8636 - val_loss: 0.3396 -

Epoch 135/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3377 - accuracy: 0.8693 - val_loss: 0.3266 - val_accuracy: 0.8696
Epoch 136/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3369 - accuracy: 0.8695 - val_loss: 0.3329 - val_accuracy: 0.8704
Epoch 137/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3356 - accuracy: 0.8709 - val_loss: 0.3245 - val_accuracy: 0.8723
Epoch 138/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3419 - accuracy: 0.8681 - val_loss: 0.3832 - val_accuracy: 0.8437
Epoch 139/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3475 - accuracy: 0.8651 - val_loss: 0.3429 - val_accuracy: 0.8646
Epoch 140/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3414 - accuracy: 0.8680 - val_loss: 0.3420 - val_accuracy: 0.8689
Epoch 141/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3374 - accuracy: 0.8706 - val_loss: 0.3189 -

...

Epoch 375/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2972 - accuracy: 0.8877 - val_loss: 0.2921 - val_accuracy: 0.8858
Epoch 376/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3004 - accuracy: 0.8864 - val_loss: 0.3129 - val_accuracy: 0.8744
Epoch 377/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2951 - accuracy: 0.8869 - val_loss: 0.2903 - val_accuracy: 0.8882
Epoch 378/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2924 - accuracy: 0.8902 - val_loss: 0.3137 - val_accuracy: 0.8749
Epoch 379/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3029 - accuracy: 0.8849 - val_loss: 0.2769 - val_accuracy: 0.8954
Epoch 380/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2924 - accuracy: 0.8904 - val_loss: 0.2880 - val_accuracy: 0.8900
Epoch 381/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3052 - accuracy: 0.8854 - val_loss: 0.2998 -

Epoch 402/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2946 - accuracy: 0.8889 - val_loss: 0.2802 - val_accuracy: 0.8927
Epoch 403/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2951 - accuracy: 0.8879 - val_loss: 0.2809 - val_accuracy: 0.8912
Epoch 404/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2951 - accuracy: 0.8885 - val_loss: 0.2922 - val_accuracy: 0.8843
Epoch 405/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2987 - accuracy: 0.8874 - val_loss: 0.2990 - val_accuracy: 0.8830
Epoch 406/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2993 - accuracy: 0.8885 - val_loss: 0.2835 - val_accuracy: 0.8891
Epoch 407/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2958 - accuracy: 0.8874 - val_loss: 0.2773 - val_accuracy: 0.8953
Epoch 408/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3052 - accuracy: 0.8843 - val_loss: 0.2867 -

Epoch 429/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2961 - accuracy: 0.8873 - val_loss: 0.2764 - val_accuracy: 0.8961
Epoch 430/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2934 - accuracy: 0.8892 - val_loss: 0.3112 - val_accuracy: 0.8782
Epoch 431/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2849 - accuracy: 0.8932 - val_loss: 0.2804 - val_accuracy: 0.8923
Epoch 432/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2972 - accuracy: 0.8880 - val_loss: 0.2960 - val_accuracy: 0.8845
Epoch 433/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2930 - accuracy: 0.8894 - val_loss: 0.2927 - val_accuracy: 0.8864
Epoch 434/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2930 - accuracy: 0.8911 - val_loss: 0.3178 - val_accuracy: 0.8758
Epoch 435/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2929 - accuracy: 0.8882 - val_loss: 0.2913 -

Epoch 456/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2929 - accuracy: 0.8882 - val_loss: 0.2776 - val_accuracy: 0.8907
Epoch 457/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2952 - accuracy: 0.8874 - val_loss: 0.2742 - val_accuracy: 0.8951
Epoch 458/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2926 - accuracy: 0.8903 - val_loss: 0.2759 - val_accuracy: 0.8943
Epoch 459/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2859 - accuracy: 0.8930 - val_loss: 0.2819 - val_accuracy: 0.8912
Epoch 460/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2905 - accuracy: 0.8905 - val_loss: 0.2878 - val_accuracy: 0.8903
Epoch 461/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2971 - accuracy: 0.8871 - val_loss: 0.3149 - val_accuracy: 0.8794
Epoch 462/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3001 - accuracy: 0.8868 - val_loss: 0.2828 -

Epoch 483/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2966 - accuracy: 0.8882 - val_loss: 0.2827 - val_accuracy: 0.8882
Epoch 484/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2919 - accuracy: 0.8890 - val_loss: 0.2971 - val_accuracy: 0.8849
Epoch 485/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2872 - accuracy: 0.8916 - val_loss: 0.2958 - val_accuracy: 0.8846
Epoch 486/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2852 - accuracy: 0.8927 - val_loss: 0.2982 - val_accuracy: 0.8841
Epoch 487/500
173/173 [==============================] - 2s 13ms/step - loss: 0.3090 - accuracy: 0.8835 - val_loss: 0.2793 - val_accuracy: 0.8927
Epoch 488/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2871 - accuracy: 0.8922 - val_loss: 0.2727 - val_accuracy: 0.8937
Epoch 489/500
173/173 [==============================] - 2s 13ms/step - loss: 0.2836 - accuracy: 0.8922 - val_loss: 0.2832 -

In [15]:
dfanalysis=pd.read_csv("cnntrainanalysisGPSwithScaling.csv")
dfanalysis.describe()

,epoch,accuracy,loss,val_accuracy,val_loss
count,500.000000,500.000000,500.000000,500.000000,500.000000
mean,249.500000,0.865969,0.346399,0.867759,0.336133
std,144.481833,0.042345,0.097651,0.040156,0.090818
min,0.000000,0.605948,0.283602,0.644844,0.271838
25%,124.750000,0.865519,0.298551,0.866859,0.291675
50%,249.500000,0.879806,0.314478,0.879410,0.309394
75%,374.250000,0.886799,0.347523,0.887653,0.336470
max,499.000000,0.893216,1.146796,0.900407,0.886518


In [16]:
from keras.models import load_model
model = load_model('cnn_modelGPSwithScaling.hdf5')

In [17]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))

461/461 [==============================] - 2s 2ms/step - loss: 0.2830 - accuracy: 0.8910

Loss: 0.28, Accuracy: 89.10%


In [18]:
y_pred=model.predict(X_test) 
#y_pred=np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)
#y_pred=model.predict(X_test) 
#y_pred=np.argmax(y_pred,axis=1)
#y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average=None)
precision = precision_score(y_test, y_pred , average=None)
f1 = f1_score(y_test, y_pred, average=None)

In [19]:
recallList=recall.tolist()
precisionList=precision.tolist()
f1lList=f1.tolist()

In [20]:
name_class=['normal','backdoor','ddos', 'password',  'injection', 'ransomware',  'xss' ,'scanning']
name_class.sort()
name_class

['backdoor',
 'ddos',
 'injection',
 'normal',
 'password',
 'ransomware',
 'scanning',
 'xss']

In [21]:
recallDict={i:j for i,j in zip(name_class,recallList)}
precisionDict={i:j for i,j in zip(name_class,precisionList)}
f1Dict={i:j for i,j in zip(name_class,f1lList)}

## Average Recall

In [22]:
recallDict

{'backdoor': 0.979594163246694,
 'ddos': 0.8192771084337349,
 'injection': 0.7569721115537849,
 'normal': 0.5779060181368508,
 'password': 0.8015686274509803,
 'ransomware': 0.6068376068376068,
 'scanning': 0.25735294117647056,
 'xss': 0.9154929577464789}

## Average Precision

In [23]:
precisionDict

{'backdoor': 0.9589331547818324,
 'ddos': 0.8012568735271013,
 'injection': 0.8497316636851521,
 'normal': 0.7947845804988662,
 'password': 0.8538011695906432,
 'ransomware': 0.8402366863905325,
 'scanning': 0.546875,
 'xss': 0.9848484848484849}

## Average F1 score

In [24]:
f1Dict

{'backdoor': 0.9691535555179609,
 'ddos': 0.8101667990468625,
 'injection': 0.8006742520016855,
 'normal': 0.6692124105011934,
 'password': 0.8268608414239481,
 'ransomware': 0.7047146401985112,
 'scanning': 0.35,
 'xss': 0.948905109489051}

In [25]:
test_loss, test_acc = cnn.evaluate(X_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

461/461 [==============================] - 1s 2ms/step - loss: 0.2830 - accuracy: 0.8910
Test accuracy 0.8910447955131531
Test loss 0.28295716643333435


In [26]:
train_loss, train_acc = cnn.evaluate(X_train, y_train)

print("Train accuracy", train_acc)
print("Train loss", train_loss)

1382/1382 [==============================] - 3s 2ms/step - loss: 0.2639 - accuracy: 0.9045
Train accuracy 0.9045228362083435
Train loss 0.26385802030563354
